In [ ]:
from functools import partial

from keras.models import Model
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Concatenate
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import Input
from keras.layers import Lambda
from keras.layers import MaxPooling2D
from keras.layers import add
from keras import backend as K


def scaling(x, scale):
    return x * scale


def conv2d_bn(x,
              filters,
              kernel_size,
              strides=1,
              padding='same',
              activation='relu',
              use_bias=False,
              name=None):
    x = Conv2D(filters,
               kernel_size,
               strides=strides,
               padding=padding,
               use_bias=use_bias,
               name=name)(x)
    if not use_bias:
        bn_axis = 1 if K.image_data_format() == 'channels_first' else 3
        bn_name = _generate_layer_name('BatchNorm', prefix=name)
        x = BatchNormalization(axis=bn_axis, momentum=0.995, epsilon=0.001,
                               scale=False, name=bn_name)(x)
    if activation is not None:
        ac_name = _generate_layer_name('Activation', prefix=name)
        x = Activation(activation, name=ac_name)(x)
    return x


def _generate_layer_name(name, branch_idx=None, prefix=None):
    if prefix is None:
        return None
    if branch_idx is None:
        return '_'.join((prefix, name))
    return '_'.join((prefix, 'Branch', str(branch_idx), name))


def _inception_resnet_block(x, scale, block_type, block_idx, activation='relu'):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else 3
    if block_idx is None:
        prefix = None
    else:
        prefix = '_'.join((block_type, str(block_idx)))
    name_fmt = partial(_generate_layer_name, prefix=prefix)

    if block_type == 'Block35':
        branch_0 = conv2d_bn(x, 32, 1, name=name_fmt('Conv2d_1x1', 0))
        branch_1 = conv2d_bn(x, 32, 1, name=name_fmt('Conv2d_0a_1x1', 1))
        branch_1 = conv2d_bn(branch_1, 32, 3, name=name_fmt('Conv2d_0b_3x3', 1))
        branch_2 = conv2d_bn(x, 32, 1, name=name_fmt('Conv2d_0a_1x1', 2))
        branch_2 = conv2d_bn(branch_2, 32, 3, name=name_fmt('Conv2d_0b_3x3', 2))
        branch_2 = conv2d_bn(branch_2, 32, 3, name=name_fmt('Conv2d_0c_3x3', 2))
        branches = [branch_0, branch_1, branch_2]
    elif block_type == 'Block17':
        branch_0 = conv2d_bn(x, 128, 1, name=name_fmt('Conv2d_1x1', 0))
        branch_1 = conv2d_bn(x, 128, 1, name=name_fmt('Conv2d_0a_1x1', 1))
        branch_1 = conv2d_bn(branch_1, 128, [1, 7], name=name_fmt('Conv2d_0b_1x7', 1))
        branch_1 = conv2d_bn(branch_1, 128, [7, 1], name=name_fmt('Conv2d_0c_7x1', 1))
        branches = [branch_0, branch_1]
    elif block_type == 'Block8':
        branch_0 = conv2d_bn(x, 192, 1, name=name_fmt('Conv2d_1x1', 0))
        branch_1 = conv2d_bn(x, 192, 1, name=name_fmt('Conv2d_0a_1x1', 1))
        branch_1 = conv2d_bn(branch_1, 192, [1, 3], name=name_fmt('Conv2d_0b_1x3', 1))
        branch_1 = conv2d_bn(branch_1, 192, [3, 1], name=name_fmt('Conv2d_0c_3x1', 1))
        branches = [branch_0, branch_1]
    else:
        raise ValueError('Unknown Inception-ResNet block type. '
                         'Expects "Block35", "Block17" or "Block8", '
                         'but got: ' + str(block_type))

    mixed = Concatenate(axis=channel_axis, name=name_fmt('Concatenate'))(branches)
    up = conv2d_bn(mixed,
                   K.int_shape(x)[channel_axis],
                   1,
                   activation=None,
                   use_bias=True,
                   name=name_fmt('Conv2d_1x1'))
    up = Lambda(scaling,
                output_shape=K.int_shape(up)[1:],
                arguments={'scale': scale})(up)
    x = add([x, up])
    if activation is not None:
        x = Activation(activation, name=name_fmt('Activation'))(x)
    return x


def InceptionResNetV1(input_shape=(160, 160, 3),
                      classes=128,
                      dropout_keep_prob=0.8,
                      weights_path=None):
    inputs = Input(shape=input_shape)
    x = conv2d_bn(inputs, 32, 3, strides=2, padding='valid', name='Conv2d_1a_3x3')
    x = conv2d_bn(x, 32, 3, padding='valid', name='Conv2d_2a_3x3')
    x = conv2d_bn(x, 64, 3, name='Conv2d_2b_3x3')
    x = MaxPooling2D(3, strides=2, name='MaxPool_3a_3x3')(x)
    x = conv2d_bn(x, 80, 1, padding='valid', name='Conv2d_3b_1x1')
    x = conv2d_bn(x, 192, 3, padding='valid', name='Conv2d_4a_3x3')
    x = conv2d_bn(x, 256, 3, strides=2, padding='valid', name='Conv2d_4b_3x3')

    # 5x Block35 (Inception-ResNet-A block):
    for block_idx in range(1, 6):
        x = _inception_resnet_block(x,
                                    scale=0.17,
                                    block_type='Block35',
                                    block_idx=block_idx)
      # Mixed 6a (Reduction-A block):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else 3
    name_fmt = partial(_generate_layer_name, prefix='Mixed_6a')
    branch_0 = conv2d_bn(x,
                         384,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 0))
    branch_1 = conv2d_bn(x, 192, 1, name=name_fmt('Conv2d_0a_1x1', 1))
    branch_1 = conv2d_bn(branch_1, 192, 3, name=name_fmt('Conv2d_0b_3x3', 1))
    branch_1 = conv2d_bn(branch_1,
                         256,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 1))
    branch_pool = MaxPooling2D(3,
                               strides=2,
                               padding='valid',
                               name=name_fmt('MaxPool_1a_3x3', 2))(x)
    branches = [branch_0, branch_1, branch_pool]
    x = Concatenate(axis=channel_axis, name='Mixed_6a')(branches)

    # 10x Block17 (Inception-ResNet-B block):
    for block_idx in range(1, 11):
        x = _inception_resnet_block(x,
                                    scale=0.1,
                                    block_type='Block17',
                                    block_idx=block_idx)

    # Mixed 7a (Reduction-B block): 8 x 8 x 2080
    name_fmt = partial(_generate_layer_name, prefix='Mixed_7a')
    branch_0 = conv2d_bn(x, 256, 1, name=name_fmt('Conv2d_0a_1x1', 0))
    branch_0 = conv2d_bn(branch_0,
                         384,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 0))
    branch_1 = conv2d_bn(x, 256, 1, name=name_fmt('Conv2d_0a_1x1', 1))
    branch_1 = conv2d_bn(branch_1,
                         256,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 1))
    branch_2 = conv2d_bn(x, 256, 1, name=name_fmt('Conv2d_0a_1x1', 2))
    branch_2 = conv2d_bn(branch_2, 256, 3, name=name_fmt('Conv2d_0b_3x3', 2))
    branch_2 = conv2d_bn(branch_2,
                         256,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 2))
    branch_pool = MaxPooling2D(3,
                               strides=2,
                               padding='valid',
                               name=name_fmt('MaxPool_1a_3x3', 3))(x)
    branches = [branch_0, branch_1, branch_2, branch_pool]
    x = Concatenate(axis=channel_axis, name='Mixed_7a')(branches)

    # 5x Block8 (Inception-ResNet-C block):
    for block_idx in range(1, 6):
        x = _inception_resnet_block(x,
                                    scale=0.2,
                                    block_type='Block8',
                                    block_idx=block_idx)
    x = _inception_resnet_block(x,
                                scale=1.,
                                activation=None,
                                block_type='Block8',
                                block_idx=6)

    # Classification block
    x = GlobalAveragePooling2D(name='AvgPool')(x)
    x = Dropout(1.0 - dropout_keep_prob, name='Dropout')(x)
    # Bottleneck
    x = Dense(classes, use_bias=False, name='Bottleneck')(x)
    bn_name = _generate_layer_name('BatchNorm', prefix='Bottleneck')
    x = BatchNormalization(momentum=0.995, epsilon=0.001, scale=False,
                           name=bn_name)(x)

    # Create model
    model = Model(inputs, x, name='inception_resnet_v1')
    if weights_path is not None:
        model.load_weights(weights_path)

    return model

In [ ]:
# Initializing the model.
embeddings_generator = InceptionResNetV1(
        input_shape=(None, None, 3),
        classes=128,
    )
# Loading the prebuilt weights.
embeddings_generator.load_weights('facenet_keras_weights.h5')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import cv2
import os
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from sklearn.preprocessing import LabelEncoder,Normalizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from IPython.display import FileLink
import joblib

In [ ]:
def load_images_and_labels(dir_path,model):
    """
    Parameters
    ----------
    dir_path : base directory path that contains the folders of each actor.
    model : facenet model that is loaded with trained weights. (Transfer learning)
    
    Functionality
    -------------
    1. Function reads all the images in the base directory by traversing recursively across all folders.
    2. All the imagees are standardized and the 128 face embeddings will be generated for all the images.
    
    Output
    ------
    face_pixels : list of numpy arrays where each numpy array represents an image (face).
    face_embeddings : list of embeddings for all the images read.
    labels : list of labels where each label represents the name of the actor
    """
    face_pixels,face_embeddings,labels=[],[],[]
    for actor_dir in tqdm(os.listdir(dir_path)):
        for actor_image in os.listdir(dir_path+actor_dir):
            # reading the image
            img=cv2.imread(dir_path+actor_dir+'/'+actor_image)
            
            # converting the image to RGB format
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            
            # Resizing the image to the size required by Facenet
            img=cv2.resize(img,(160,160))
            img = img.astype('float32')
            
            # Normalizing the images
            mean, std = img.mean(), img.std()
            img = (img - mean) / std
            face_pixels.append(img)
            labels.append(actor_dir)
            img = np.expand_dims(img, axis=0)
            
            # Getting the face embeddings from the model.
            face_embeddings.append(model.predict(img))
    return face_pixels,face_embeddings,labels

In [ ]:
base_dir='./de/'
face_pixels,face_embeddings,labels=load_images_and_labels(base_dir,embeddings_generator)

In [ ]:
# converting the lists into numpy arrays
face_embeddings=np.array(face_embeddings)
face_pixels=np.array(face_pixels)
labels=np.array(labels)
face_embeddings=face_embeddings.reshape(face_embeddings.shape[0],face_embeddings.shape[2])

In [ ]:
# saving these files to overcome processing time of the dataset
np.savez_compressed('datas/data.npz',a=face_embeddings,b=face_pixels,c=labels)
FileLink(r'data.npz')

In [ ]:
# loading back the generated embeddings, face images and labels
data=np.load('datas/data.npz')
face_embeddings,face_pixels,labels=data['a'],data['b'],data['c']

CHECKING EMBEDDING AUTHENTICITY

In [ ]:
# check the authenticity of the generated emebeddings
temp=pd.DataFrame(face_embeddings)
print("Total embeddings generated: ",len(temp))
print("Total embeddings after dropping duplicates: ",len(temp.drop_duplicates()))

In [ ]:
# Fitting a label encoder of names of actors
label_encoder=LabelEncoder()
encoded_labels=label_encoder.fit_transform(labels)

# storing the label encoder object for using in deplyment environment
joblib.dump(label_encoder, 'datas/label_encoder.sav')

# Downloading the picked label encoder file
FileLink(r'label_encoder.sav')

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(face_embeddings,encoded_labels,test_size=0.3,stratify=encoded_labels)

In [ ]:
print(f"Shape of Xtrain : {x_train.shape}")
print(f"Shape of Ytrain : {y_train.shape}")
print(f"Shape of Xtest : {x_test.shape}")
print(f"Shape of Ytest : {y_test.shape}")

In [ ]:
model = SVC(probability=True)
model.fit(x_train,y_train )

In [ ]:
yhat_train = model.predict(x_train)
yhat_test = model.predict(x_test)

MODEL EVALUATION

In [ ]:
# calculating accuracy score on train set
score_train = accuracy_score(y_train, yhat_train)

# calculating accuracy score on test set
score_test = accuracy_score(y_test, yhat_test)

print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

In [ ]:
# Finding out probabilites of first test sample for every target class
probs_svc = model.decision_function([x_test[0]])
probs_svc = (probs_svc - probs_svc.min()) / (probs_svc.max() - probs_svc.min())
probs_svc

In [ ]:
print(classification_report(yhat_test,y_test,target_names=label_encoder.classes_))

In [ ]:
print(confusion_matrix(yhat_test,y_test))

In [ ]:
joblib.dump(model, 'datas/svm_classifier.sav')
FileLink(r'svm_classifier.sav')

In [ ]:
random_index=np.random.choice(len(labels))
predicted_label=label_encoder.inverse_transform(model.predict([face_embeddings[random_index]]))[0]
actual_label=labels[random_index]
plt.title(f"Actual Label: {actual_label} ")
plt.xlabel(f"Predicted Label: {predicted_label}")
plt.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
plt.imshow(face_pixels[random_index])
plt.show()

In [ ]:
# Saving one embedding for each face
mapping_dict = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
known_face_embeddings=[]
for key in mapping_dict:
    match_indices=np.where(labels==key)
    first_match_index=match_indices[0][0]
    known_face_embeddings.append(face_embeddings[first_match_index])
known_face_embeddings=np.array(known_face_embeddings)
np.savez_compressed('datas/data.npz',a=known_face_embeddings)
FileLink(r'data.npz')

In [ ]:
model

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
import joblib
from scipy.spatial.distance import cosine

# Load the pre-trained SVM model and label encoder
model = joblib.load('datas/svm_classifier.sav')
label_encoder = joblib.load('datas/label_encoder.sav')

# Load the face embeddings of known faces
known_face_data = np.load('datas/data.npz')
known_face_embeddings = known_face_data['a']

# Load the face recognition model
# Initializing the model.
embeddings_generator = InceptionResNetV1(
        input_shape=(None, None, 3),
        classes=128,
    )
# Loading the prebuilt weights.
embeddings_generator.load_weights('facenet_keras_weights.h5')
def recognize_faces(frame, threshold=0.7):
    # Detect faces in the frame
    faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml').detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        # Extract the face region
        face = frame[y:y+h, x:x+w]
        
        # Preprocess the face for recognition
        face = cv2.resize(face, (160, 160))
        face = (face.astype('float32') - 127.5) / 128.0  # Normalize to the range [-1, 1]
        face = np.expand_dims(face, axis=0)
        
        # Generate face embeddings using the pre-trained model
        face_embedding = embeddings_generator.predict(face).flatten()
        
        # Calculate cosine similarity between the face and known faces
        similarities = [1 - cosine(face_embedding, known_face) for known_face in known_face_embeddings]
        print(similarities)
        # Find the maximum similarity
        max_similarity = max(similarities)
        
        # Verify if the face is similar enough to any known face
        if max_similarity >= threshold:
            # Get the label corresponding to the most similar known face
            predicted_label = label_encoder.classes_[np.argmax(similarities)]
            
            # Draw the bounding box and label on the frame
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, predicted_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return frame

# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    
    # Perform face recognition on the frame
    frame = recognize_faces(frame)
    
    # Display the frame
    cv2.imshow('Face Recognition', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the OpenCV window
cap.release()
cv2.destroyAllWindows()


In [ ]:
# Initializing the model.
embeddings_generator = InceptionResNetV1(
        input_shape=(None, None, 3),
        classes=128,
    )
# Loading the prebuilt weights.
embeddings_generator.load_weights('facenet_keras_weights.h5')

In [27]:
# Save the model architecture to JSON file
model_json = embeddings_generator.to_json()
with open("datas/model_architecture.json", "w") as json_file:
    json_file.write(model_json)

# Save the model weights to an HDF5 file
embeddings_generator.save_weights("datas/model_weights.h5")


In [28]:
import joblib

# Save the label encoder
joblib.dump(label_encoder, 'datas/label_encoder.sav')


['da/label_encoder.sav']

In [29]:
joblib.dump(model, 'datas/svm_classifier.sav')


['da/svm_classifier.sav']

In [30]:
from keras.models import load_model, Model
from keras.optimizers import Adam

# Assuming 'embeddings_generator' is your Keras model
# Compile your model before saving it
embeddings_generator.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Save the model to an HDF5 file
model_path = 'datas/your_model.h5'
embeddings_generator.save(model_path)

# Load the model on a new machine
loaded_model = load_model(model_path)


c:\Users\aarvm\miniconda3\envs\py311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Setup paths
POS_PATH_R = os.path.join('de', 'p1')
POS_PATH_K= os.path.join('de', 'p2')
POS_PATH_S = os.path.join('de' , 'p3')

In [ ]:
r_counter = 1
k_counter = 1
# Establish a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250px
    frame = frame[:,:, :]
    
    # Collect anchors 
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Create the unique file path 
        imgname = os.path.join(POS_PATH_S, 'S{:03d}.jpg'.format(r_counter))
        # Increment the counter for R
        r_counter += 1
        # Write out positive image for R
        cv2.imwrite(imgname, frame)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path 
        imgname = os.path.join(POS_PATH_K, 'K{:03d}.jpg'.format(k_counter))
        # Increment the counter for K
        k_counter += 1
        # Write out positive image for K
        cv2.imwrite(imgname, frame)

        # Show image back to screen
    cv2.imshow('Image Collection', frame)
    
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()